In [43]:
# Подключаем библиотеки
import mysql.connector
import re
from google_ngram_downloader import readline_google_store # библиотека для выгрузки нграм
from pymystem3 import Mystem
import pandas as pd

In [ ]:
# создаем соединение с сервером
password = input("Enter your password: ")
con=mysql.connector.connect(host="localhost", database="google_ngrams", user="root", password=password)

In [46]:
# создаем курсор для отправки запросов и получения результатов
cur=con.cursor(dictionary=True)

#### Как выглядят данные, которые мы будем класть в базу данных

In [72]:
fname, url, records = next(readline_google_store(ngram_len=1, lang='rus', indices='a'))
record = next(records)
request = "({}, {}, {}, {})".format(record.ngram,
                                    record.year,
                                    record.match_count,
                                    record.volume_count)
print(request)


(Allocation, 1960, 2, 1)


#### И таких все слова на заданную букву

In [73]:
# индексы униграм (символы, с которых они начинаются)
unigram_indices = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                   'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 
                   'other', 'pos', 'punctuation']

#### создадим необходимые таблицы:


In [184]:
cur.execute("CREATE TABLE `uni_grams_after_1918` (`idx` text NOT NULL, `raw_n_gram` text NOT NULL, `n_gram` text NOT NULL, `year` int NOT NULL, `match_count` int NOT NULL, `volume_count` int NOT NULL)")
cur.execute("CREATE TABLE `uni_grams_before_1918` (`idx` text NOT NULL, `raw_n_gram` text NOT NULL, `n_gram` text NOT NULL, `year` int NOT NULL, `match_count` int NOT NULL, `volume_count` int NOT NULL, `new_idx` text DEFAULT NULL, `is_bastard` boolean DEFAULT false, `new_ngram` text DEFAULT NULL)")
cur.execute("CREATE TABLE `vocabulary_before_1918` (`uni_gram` text NOT NULL, `match_count` int NOT NULL)")

#### Первая нужна для статистического анализа, вторая -  для исправления орфографии, третья - для хранения словаря дореволюционной орфографии 

In [121]:
# функция,выгружающая нграмы до и после 1918 года в разные таблицы
# на вход подаются параметры, определяющие длину нграм, язык и индекс

def load_ngrams(table_name, my_len, my_lang, my_indices, before_1918=False):
    fname, url, records = next(readline_google_store(ngram_len=my_len, lang=my_lang, indices=my_indices))
    record = next(records)
    e = 0
    while True:
        try:
            if before_1918:
                if record.year >= 1918:
                    record = next(records)
                elif record.year < 1918:
                    ngram = record.ngram
                    request = "INSERT INTO " + table_name + "(idx, raw_n_gram, n_gram, year, match_count, volume_count)"
                    request += " VALUES ('{}', '{}', '{}', {}, {}, {});".format(my_indices,
                                                             ngram.replace("'","\\\'"),
                                                             normalize(ngram).replace("'","\\\'"),
                                                             record.year,
                                                             record.match_count,
                                                             record.volume_count)
                    cur.execute(request)
                    con.commit()
                    if e%1000==0:
                        print('loaded: ' + str(e))  # чтобы видеть количество загрузок
                    e += 1
                    record = next(records)
            else:
                if record.year < 1918:
                    record = next(records)
                elif record.year >= 1918:
                    ngram = record.ngram
                    request = "INSERT INTO " + table_name + "(idx, raw_n_gram, n_gram, year, match_count, volume_count)"
                    request += " VALUES ('{}', '{}', '{}', {}, {}, {});".format(my_indices,
                                                                 ngram.replace("'","\\\'"),
                                                                 normalize(ngram).replace("'", "\\\'"),
                                                                 record.year,
                                                                 record.match_count,
                                                                 record.volume_count)
                    cur.execute(request)
                    con.commit()
                    if e%1000==0:
                        print('loaded: ' + str(e)) # чтобы видеть количество загрузок
                    e += 1
                    record = next(records)
        except StopIteration:
            break
            print("StopIteration")
    
# функция, нормализующая нграмы:
# избавление от тэгов
# замена нулей в словах на букву "о"
# замена единиц в словах на букву "i"

def normalize(ngram):
    if '_' in ngram:
        index = ngram.find('_')
        ngram = ngram[: index]
#   if not re.sub("[.,:\'-]", '', ngram).isdigit():
#     ngram = re.sub("0", 'О', ngram)
#     ngram = re.sub("1", 'i', ngram)
    return(ngram)

##### Загрузка данных в первую таблицу

In [ ]:
%%time
load_ngrams("uni_grams_before_1918",1, 'rus', 'a', before_1918=True)

##### Загрузка данных во вторую таблицу

In [ ]:
%%time
load_ngrams('uni_grams_after_1918', 1, 'rus', 'a', before_1918=False)

#### 3. Создадим словарь дорефолюционного русского языка. Для этого возьмем имеющийся корпус текстов с нужной орфографией, полученный с данных викитеки

In [2]:
import nltk
from nltk import punkt # для сегментации предложений
from nltk.tokenize import sent_tokenize
import re #для замены ненужных символов
from collections import Counter

In [4]:
class VocabMaker(object):
    def __init__(self, corpus, min_freq=5):
        self.corpus = self.read_corpus(corpus)
        self.corpus_words, self.num_corpus_words, self.counts = self.distinct_words(min_freq)
        self.index2word, self.word2index = self.build_index()
               
    def read_corpus(self, filename, min_freq=20):
        print('Reading a corpus')
        if str(filename).endswith('.txt'):
            file = open(filename, 'r', encoding = 'utf-8')
            return([[ re.sub('{[^}]}', '', w.lower()) for w in nltk.tokenize.WordPunctTokenizer().tokenize(sentence)] 
                    for sentence in sent_tokenize(file.read(), language='slovene')])
            file.close()
            print('Done!')
        else:
            return(filename)
    
    def distinct_words(self, min_freq):
        print('Estimating cospus size')
        flat_corpus = [word for sentence in self.corpus for word in sentence]
        counts = Counter(flat_corpus)
        corpus_words = sorted(list(set([word for word in flat_corpus])))
        num_corpus_words = len(corpus_words)
        return(corpus_words, num_corpus_words, counts)
        print('Done!')
        print('You can checkout the number of corpus words and the words themselves with commands .num_corpus_words, .corpus_words')
    
    def build_index(self):
        word2index = dict()
        for word in self.corpus_words:
            word2index[word] = len(word2index)
            word2index['UNK'] = 0
        return ({index:word for word, index in word2index.items()}, word2index)

In [5]:
VM = VocabMaker("corpus.txt")

Reading a corpus
Estimating cospus size


In [10]:
VM.corpus[1]

['спасъ',
 'на',
 'бору',
 ',',
 'въ',
 'московскомъ',
 'кремлѣ',
 '/',
 'до',
 '358011',
 'о',
 'красотѣ',
 'бога',
 '(',
 'вивекананда',
 '/',
 'рудникова',
 ')/',
 'до',
 '434173',
 'о',
 'верхне',
 '-',
 'иргизском',
 'монастырѣ',
 ',',
 'о',
 'пріѣздѣ',
 'въ',
 'него',
 'генерала',
 'рунича',
 ',',
 'пожалованіи',
 'императора',
 'павла',
 '1',
 '-',
 'го',
 'на',
 'возобновленіе',
 'сгорѣвшей',
 'в',
 'немъ',
 'церкви',
 'къ',
 'числу',
 'любопытныхъ',
 'историческихъ',
 'воспоминаній',
 'о',
 'томъ',
 'времени',
 'должно',
 'отнести',
 'пребываніе',
 'въ',
 'петергофѣ',
 'лѣтомъ',
 '1797',
 'года',
 'бывшаго',
 'польскаго',
 'короля',
 'станислава',
 '-',
 'августа',
 '?']

In [6]:
VM.num_corpus_words

475317

In [8]:
len(VM.corpus)

1116367

In [209]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'

In [218]:
flat_corpus = [word.strip(punct) for sentence in VM.corpus for word in sentence if word]

In [200]:
def generate_ngrams(corpus, n):
    # Склеивание токенов в нграмы
    ngrams = zip(*[corpus[i:] for i in range(n)])
    joined = [" ".join(ngram) for ngram in ngrams]
    return joined

In [212]:
generate_ngrams(flat_corpus[:10], 5)

['41382 письмо ѳёдора михайловича достоевскаго',
 'письмо ѳёдора михайловича достоевскаго любови',
 'ѳёдора михайловича достоевскаго любови ѳедоровнѣ',
 'михайловича достоевскаго любови ѳедоровнѣ достоевской',
 'достоевскаго любови ѳедоровнѣ достоевской 25',
 'любови ѳедоровнѣ достоевской 25 ']

#### Воспользуемся сначала только униграмами

In [219]:
uni_grams = generate_ngrams(flat_corpus, 1)
counts = Counter(uni_grams)
unique_uni_grams = sorted(list(set([word for word in uni_grams])))

In [222]:
len(unique_uni_grams)

474279

#### Загрузим данные в базу

In [227]:
%%time
e=0
for word in uni_grams:
    word.replace('"', '\\"')
    request = "INSERT INTO vocabulary_before_1918(uni_gram, match_count)"
    request += " VALUES ('{}', {});".format(word.replace("'","\\\'"),
                                            counts[word])
    cur.execute(request)
    con.commit()
    if e%1000==0:
        print('loaded: ' + str(e)) # чтобы видеть количество загрузок
    e += 1

####  Отметим нграмы из "uni_grams_before_1918", не соответсвующие значениям словаря


In [ ]:
%%time
# Меняем дефолтное значение в колонке "is_bastard" если слова нет в словаре

req="""UPDATE uni_grams_before_1918
SET is_bastard=true
WHERE n_gram NOT IN 
(SELECT uni_gram FROM vocabulary_before_1918)"""

In [ ]:
cur.execute(req)
present=cur.fetchall()
con.commit()